In [185]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import PolynomialFeatures
import warnings
import category_encoders as ce
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.exceptions import DataConversionWarning
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_regression,f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
import lightgbm as lgb

pd.options.display.max_columns = 100

In [186]:
train_features = pd.read_csv('train_features.csv')
test_features = pd.read_csv('test_features.csv')
train_labels = pd.read_csv('train_labels.csv')
sample_submission = pd.read_csv('sample_submission.csv')

train_features.shape, test_features.shape, train_labels.shape, sample_submission.shape

((59400, 40), (14358, 40), (59400, 2), (14358, 2))

In [187]:
train = train_features.copy()

In [188]:
year_mean = train[train['construction_year']>0]['construction_year'].mean()
year_mean = round(year_mean)

test_year_mean = test_features[test_features['construction_year']>0]['construction_year'].mean()
test_year_mean = round(test_year_mean)


train.loc[train['construction_year']==0, 'construction_year'] = int(year_mean)
test_features.loc[test_features['construction_year']==0,'construction_year'] = int(test_year_mean)

In [189]:
import random

def random_std(year):

    return year - random.randint(-10,10)

def random_tsh(amount):

    return amount + random.uniform(-1062.35, 1957.82)

In [190]:
random_tsh(200)

-728.846353589262

In [191]:
train_tsh_mean = train[train['amount_tsh']>0]['amount_tsh'].mean()
test_tsh_mean = test_features[test_features['amount_tsh']>0]['amount_tsh'].mean()

train.loc[train['amount_tsh']==0,'amount_tsh']=float(train_tsh_mean)
test_features.loc[test_features['amount_tsh']==0,'amount_tsh']=float(test_tsh_mean)

In [192]:
# train.loc[train['construction_year']==1997,'construction_year'].apply(random_std(),axis=1)
# test_features.loc[test_features['construction_year']==1997,'construction_year'].apply(random_std(),axis=1)
# train.loc[train['amount_tsh']==train_tsh_mean, 'amount_tsh'].apply(random_tsh(),axis=1)
# test_features.loc[test_features['amount_tsh']==test_tsh_mean, 'amount_tsh'].apply(random_tsh(),axis=1)

# train.shape, test_features.shape

In [193]:
# train.loc[train['construction_year']==1997,'construction_year'].apply(random_std)
# test_features.loc[test_features['construction_year']==1997,'construction_year'].transform(random_std)
# train.loc[train['amount_tsh']==train_tsh_mean, 'amount_tsh'].transform(random_tsh)
# test_features.loc[test_features['amount_tsh']==test_tsh_mean, 'amount_tsh'].transform(random_tsh)

# train.shape, test_features.shape

In [194]:
train.describe(exclude=np.number)

,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,recorded_by,scheme_management,scheme_name,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
count,59400,55765,55745,59400,59400,59029,59400,59400,59400,56066,59400,55523,31234,56344,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400
unique,356,1897,2145,37400,9,19287,21,125,2092,2,1,12,2696,2,18,13,7,12,5,7,7,8,6,5,5,10,7,3,7,6
top,2011-03-15,Government Of Tanzania,DWE,none,Lake Victoria,Madukani,Iringa,Njombe,Igosi,True,GeoData Consultants Ltd,VWC,K,True,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
freq,572,9084,17402,3563,10248,508,5294,2503,307,51011,59400,36793,682,38852,26780,26780,26780,40507,52490,25348,25348,50818,50818,33186,33186,17021,17021,45794,28522,34625


In [195]:
train['funder'].fillna('?',inplace=True)
train['installer'].fillna('?',inplace=True)
train['subvillage'].fillna('?',inplace=True)
train['public_meeting'].fillna('?',inplace=True)
train['scheme_management'].fillna('?',inplace=True)
train['scheme_name'].fillna('?',inplace=True)
train['permit'].fillna('?',inplace=True)
test_features['funder'].fillna('?',inplace=True)
test_features['installer'].fillna('?',inplace=True)
test_features['subvillage'].fillna('?',inplace=True)
test_features['public_meeting'].fillna('?',inplace=True)
test_features['scheme_management'].fillna('?',inplace=True)
test_features['scheme_name'].fillna('?',inplace=True)
test_features['permit'].fillna('?',inplace=True)

In [196]:
train['age'] = (2019 - train['construction_year']).astype(int)
test_features['age'] = (2019 - test_features['construction_year']).astype(int)

In [197]:
train_days_since = np.array(train['date_recorded'].values, dtype='datetime64')
test_days_since = np.array(test_features['date_recorded'].values,dtype='datetime64')

train_birth = train['construction_year'].astype(str)
test_birth = test_features['construction_year'].astype(str)

In [198]:
train_years_since = np.datetime_as_string(train_days_since, unit='Y')
test_years_since = np.datetime_as_string(test_days_since, unit='Y')


In [199]:
train_years_since = train_years_since.astype(int)
test_years_since = test_years_since.astype(int)
train_birth = train_birth.astype(int)
test_birth = test_birth.astype(int)

in_train_years = []
in_test_years = []

for i in range(0,len(train_years_since)):
    x = train_years_since[i] - train_birth[i]
    in_train_years.append(x)
    

for i in range(0,len(test_years_since)):
    x = test_years_since[i] - test_birth[i]
    in_test_years.append(x)

In [200]:
train['years_until_record'] = in_train_years
test_features['years_until_record'] = in_test_years

In [201]:
drop_these=[
    'date_recorded',
    'wpt_name',
    'recorded_by',
    'lga',
    'ward',
    'scheme_name', 
    'scheme_management',
    'funder',
    'installer',
    'num_private',
    'subvillage',
    'basin',
    'longitude',
    'latitude',
    'waterpoint_type_group',
    'extraction_type_group',
    'extraction_type_class',
    'management_group', 
]
train.drop(columns=drop_these,inplace=True)
test_features.drop(columns=drop_these,inplace=True)

train.shape, test_features.shape

((59400, 24), (14358, 24))

In [202]:
str_cols = train.select_dtypes(include=[object])

In [203]:
# hashing = ce.HashingEncoder(
#     cols = list(str_cols.columns),
#     verbose=1,
#     drop_invariant=True,
#     return_df=True,
# )

# train_features = hashing.fit_transform(train)
# test_features = hashing.fit_transform(test_features)

# train_features.shape, test_features.shape

In [204]:
# train_features

In [205]:
oe = ce.OrdinalEncoder(verbose=1,mapping=None,cols=None,drop_invariant=True,
                       return_df=True, impute_missing=True,handle_unknown='ignore')

train_features = oe.fit_transform(train)
test_features = oe.fit_transform(test_features)

train_features.shape, test_features.shape

((59400, 24), (14358, 24))

In [ ]:
X_train = train_features
X_test = test_features
y_train = train_labels['status_group']
scaler = RobustScaler()
X_train = X_train.drop(columns='id')
X_test = X_test.drop(columns='id')
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

lgb_train = lgb.Dataset(X_train, label=y_train)

params = {'boosting_type': 'gbdt',
          'max_depth' : -1,
          'objective': 'multiclass',
          'num_leaves': 42,
          'learning_rate': 0.01,
          'max_bin': 512,
          'subsample_for_bin': 100000,
          'subsample': 1,
          'subsample_freq': 1,
          'colsample_bytree': 0.8,
          'reg_alpha': 2,
          'reg_lambda': 5,
          'min_split_gain': 0.5,
          'min_child_weight': 1,
          'min_child_samples': 25,
          'num_class' : 3,
          'metric' : 'multi_logloss'}

model = lgb.LGBMClassifier(
    boosting_type = 'gbdt',
    objective = 'multiclass',
    n_jobs = -1,
    max_depth = params['max_depth'],
    max_bin = params['max_bin'],
    subsample_for_bin = params['subsample_for_bin'],
    subsample = params['subsample'],
    subsample_freq = params['subsample_freq'],
    min_split_gain = params['min_split_gain'],
    min_child_weight = params['min_child_weight'],
    min_child_samples = params['min_child_samples'],
    
)

param_grid = {
    'boosting_type': ['gbdt', 'dart'],
    'learning_rate': [0.001,0.01],
    'num_iterations': [1000,2000],
    'num_leaves': [100,200],
    'min_child_samples': [25,50],
    'subsample_for_bin': [200000],    
    'reg_alpha': [1.2],
    'reg_lambda': [1.2],
    'min_split_gain': [0.5]
}
# hopefully this works lol
gridsearch = GridSearchCV(model, param_grid = param_grid,
                          verbose=1,cv=3,n_jobs=-1,scoring='accuracy')

gridsearch.fit(X_train,y_train)

Fitting 3 folds for each of 32 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
